# fieldset_compare

compare the vector fields of different fieldsets

mainly used for checking the differences between the plume tracker OI data vs the thredds data

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import timedelta
import math

import cartopy
import cartopy.crs as ccrs
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from parcels import FieldSet, ParticleSet, JITParticle, AdvectionRK4, ErrorCode, ParticleFile
import scipy.spatial

import pyplume.utils as utils
from pyplume.parcels_utils import get_file_info
from pyplume.plotting import get_carree_axis, get_carree_gl

In [ ]:
files = [
    get_file_info(utils.CURRENT_NETCDF_DIR / "west_coast_1km_hourly/tj_plume_2020-08_interped.nc", 1, name="tj_plume"),
    get_file_info(utils.CURRENT_NETCDF_DIR / "Tot_SDLJ_20200801.nc", 1, name="tot_sdlj"),
]

# if lat and lon dimensions are the same, the stuff will run MUCH quicker
dimensions_same = True

### comparing fieldset vectors

takes the first vector field, and checks each vector. finds the closest vector in the second vector field, and finds their angle and magnitude difference

note it might take a few minutes to run

In [ ]:
def get_nearest_index(ref, lat, lon):
    return ref["latkdtree"].query([lat])[1], ref["lonkdtree"].query([lon])[1]

In [ ]:
# set up kdtrees because im lazy
for f in files:
    f["latkdtree"] = scipy.spatial.cKDTree(np.array([f["lat"]]).T)
    f["lonkdtree"] = scipy.spatial.cKDTree(np.array([f["lon"]]).T)

In [ ]:
# kind of hardcoded
ang_diff = np.full(files[0]["xrds"]["u"].shape, np.nan, dtype=np.float32)
mag_diff = np.full(ang_diff.shape, np.nan, dtype=np.float32)

if not dimensions_same:
    for t, i, j in np.ndindex(ang_diff.shape):
        time = files[0]["xrds"]["time"].values[t]
        u = files[0]["xrds"]["u"].values[t, i, j]
        if np.isnan(u) or time not in files[1]["xrds"]["time"]:
            continue
        # OI data is missing some time values
        t_other = np.where(files[1]["xrds"]["time"].values == time)[0]
        if t_other.size == 0:
            continue
        else:
            t_other = t_other[0]
        lat = files[0]["xrds"]["lat"][i].values
        lon = files[0]["xrds"]["lon"][j].values
        lat_ind, lon_ind = get_nearest_index(files[1], lat, lon)
    #     print(utils.haversine(lat, files[1]["xrds"]["lat"].values[lat_ind], lon, files[1]["xrds"]["lon"].values[lon_ind]))
        if abs(lat - files[1]["xrds"]["lat"].values[lat_ind]) > 0.00903 or abs(lon - files[1]["xrds"]["lon"].values[lon_ind]) > 0.0107:
            continue
        v = files[0]["xrds"]["v"].values[t, i, j]
        u_other = files[1]["xrds"]["u"].values[t_other, lat_ind, lon_ind]
        v_other = files[1]["xrds"]["v"].values[t_other, lat_ind, lon_ind]
        mag = np.sqrt(u * u + v * v)
        mag_other = np.sqrt(u_other * u_other + v_other * v_other)
        dot_mag_rat = (u * u_other + v * v_other) / (mag * mag_other)
        # rounding errors cause arccos to go out of domain
        if dot_mag_rat > 1:
            ang = 0
        elif dot_mag_rat < -1:
            ang = 180
        else:
            ang = np.arccos(dot_mag_rat) * 180 / math.pi
        ang_diff[t, i, j] = ang
        mag_diff[t, i, j] = abs(mag - mag_other)
else:
    # runs mUCH MUCH quicker
    for t in range(ang_diff.shape[0]):
        time = files[0]["xrds"]["time"].values[t]
        u = files[0]["xrds"]["u"].values[t]
        v = files[0]["xrds"]["v"].values[t]
        t_other = np.where(files[1]["xrds"]["time"].values == time)[0]
        if t_other.size == 0:
            print(f"time {time} missing")
            continue
        else:
            t_other = t_other[0]
        u_other = files[1]["xrds"]["u"].values[t_other]
        v_other = files[1]["xrds"]["v"].values[t_other]
        mag = np.sqrt(u * u + v * v)
        mag_other = np.sqrt(u_other * u_other + v_other * v_other)
        dot_mag_rat = (u * u_other + v * v_other) / (mag * mag_other)
        # rounding errors cause arccos to go out of domain
        dot_mag_rat[np.where(dot_mag_rat > 1)] = 1
        dot_mag_rat[np.where(dot_mag_rat < -1)] = -1
        ang = np.arccos(dot_mag_rat) * 180 / math.pi
        ang_diff[t] = ang
        mag_diff[t] = abs(mag - mag_other)

In [ ]:
np.nanmean(ang_diff), np.nanmedian(ang_diff), np.nanmax(ang_diff)

In [ ]:
np.nanmean(mag_diff), np.nanmedian(mag_diff), np.nanmax(mag_diff)

In [ ]:
ang_means = np.nanmean(ang_diff, axis=0)
mag_means = np.nanmean(mag_diff, axis=0)

distribution of measure differences

In [ ]:
plt.hist(ang_diff[np.where(~np.isnan(ang_diff))].flatten())
plt.title("angle differences (degrees)")
plt.show()
plt.hist(mag_diff[np.where(~np.isnan(mag_diff))].flatten())
plt.title("magnitude differences (m/s)")
plt.show()

generate heatmaps of measure differences

In [ ]:
lon, lat = np.meshgrid(files[0]["xrds"]["lon"].values, files[0]["xrds"]["lat"].values)
first_time = np.datetime64(files[0]["timerng"][0], "M")
deg_diff_dir = utils.get_dir(utils.PICUTRE_DIR / "fs_comp_angle")

dom = files[0]["domain"]
dom["S"] = 32.3
ax = get_carree_axis(dom)
gl = get_carree_gl(ax)
plt.contourf(lon, lat, ang_means, np.linspace(0, np.nanmax(ang_means), 30), cmap='inferno')
plt.title(f"Avg vector degree diff ({first_time})")
plt.colorbar()
plt.savefig(deg_diff_dir / f"heat_deg_{first_time}")
plt.show()

In [ ]:
deg_diff_dir = utils.get_dir(utils.PICUTRE_DIR / "fs_comp_mag")

ax = get_carree_axis(dom)
gl = get_carree_gl(ax)
plt.contourf(lon, lat, mag_means, np.linspace(0, np.nanmax(mag_means), 20), cmap='inferno')
plt.title(f"Avg magnitude diff (m/s) ({first_time})")
plt.colorbar()
plt.savefig(deg_diff_dir / f"heat_mag_{first_time}")
plt.show()